In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='6'

import torch

print(torch.__version__)
print(torch.version.cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name)

2.1.0
11.8
cuda
True
0
<function get_device_name at 0x7f6c04810a60>


In [ ]:
import pandas as pd
import ast
from tqdm import tqdm
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration

data_path = "../../data/jeehoshin/allrecipe_dataset/"
recipes = pd.read_csv(data_path + "core-data_recipe.csv")

# Preprocess nutrition data (Use 7 major nutritions and format it as a string)
for i in range(len(recipes)):
    nutri = recipes.iloc[i]['nutritions']
    nutri_dict = ast.literal_eval(nutri)

    selected_keys = ['sugars', 'sodium', 'carbohydrates', 'calories', 'fat', 'saturatedFat', 'fiber']
    filtered_nutri = {key: nutri_dict[key] for key in selected_keys if key in nutri_dict}

    nutrition_summary = ""
    for key in selected_keys:
        value = filtered_nutri.get(key, {})
        amount = value.get('displayValue', 0)
        unit = value.get('unit', '')
        nutrition_summary += f"{key}: {amount}{unit}, "
    
    recipes.at[i, 'nutrition_summary'] = nutrition_summary

recipes.drop(columns=['cooking_directions', 'nutritions'], inplace=True)

recipes.to_csv(data_path + "core-data_recipe_processed.csv", index=False)
print("nutrition data processing completed")

nutrition data processing completed


In [ ]:
import re

cache_dir = "../../data/jeehoshin/huggingface"
model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    cache_dir=cache_dir,
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(0)

processor = AutoProcessor.from_pretrained(
    model_id,
    cache_dir=cache_dir
)

def extract_summary_with_tags(generated_text):
    matches = re.search(r"<summary>(.*)", generated_text, re.DOTALL | re.IGNORECASE)
    if matches:
        return matches.group(1).strip()
    else:
        return generated_text.strip()  # fallback

def generate_batch_summary(model, processor, data_batch, device):
    images, prompts = [], []

    for recipe_id, recipe_name, ingredients, nutritions in data_batch:
        image_path = data_path + f"core-data-images/core-data-images/{recipe_id}.jpg"
        
        try:
            image = Image.open(image_path).convert("RGB")  # Load local image
        except Exception as e:
            print(f"Failed to load image: {image_path}")
            image = Image.new("RGB", (224, 224), (255, 255, 255))  # fallback blank image
        
        images.append(image)

        prompt = f"""<image>
You are a culinary expert. Given the image and the textual information below, write a simple and concise summary of the recipe. 
Your summary should combine key visual characteristics (e.g., color, shape, texture) with notable textual details (e.g., main ingredients and nutritional traits).

Name: {recipe_name}  
Ingredients: {ingredients}  
Nutrition: {nutritions}

<summary>"""

        prompts.append(prompt)
  
    inputs = processor(text = prompts, images = images, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=200)
    decoded = processor.batch_decode(output, skip_special_tokens=True)
    
    summaries = [extract_summary_with_tags(txt) for txt in decoded]

    return summaries

# Example usage
batch_size = 8
results = []
for i in tqdm(range(0, len(recipes), batch_size)):  
    batch = recipes.iloc[i:i+batch_size][['recipe_id', 'recipe_name', 'ingredients', 'nutrition_summary']].values.tolist()
    batch_results = generate_batch_summary(model, processor, batch, device=torch.device("cuda"))

    results.extend(batch_results)

recipes['summary'] = results

recipes.drop(columns=['image_url', 'ingredients', 'nutrition_summary'], inplace=True)

recipes.to_csv(data_path + "short_recipe_summaries.csv", index=False)

print("Summaries generated and saved.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
100%|█████████████████████████████████████████████████████████████████████| 5704/5704 [12:40:32<00:00,  8.00s/it]


Summaries generated and saved.


In [1]:
import pandas as pd
summaries = pd.read_csv(data_path + "short_recipe_summaries.csv")

for i in range(20):
    print(f"Recipe id : {summaries.iloc[i]['recipe_id']}")
    print(f"Recipe name : {summaries.iloc[i]['recipe_name']}")
    print(f"Recipe summary : {summaries.iloc[i]['summary']}")
    print()

# check invalid summaries
count = summaries['summary'].isna().sum()
print(f"# of invalid summaries : {count}")

Recipe id : 240488
Recipe name : Pork Loin, Apples, and Sauerkraut
Recipe summary : A pork loin roast is seasoned with Thai seasoning, salt, garlic powder, and ground black pepper. It is then cooked with sauerkraut and sliced Granny Smith apples. The dish is served with apple cider and brown sugar. The nutritional value of the dish includes a moderate amount of sugar, sodium, and calories, with a good amount of fiber.

Recipe id : 218939
Recipe name : Foolproof Rosemary Chicken Wings
Recipe summary : The Foolproof Rosemary Chicken Wings recipe involves marinating chicken wings in a mixture of rosemary sprigs, garlic, olive oil, lemon pepper, and seasoned salt. The dish is cooked in the oven, resulting in a flavorful and tender meal. The wings are golden brown and have a slightly crispy texture, with a hint of rosemary and garlic flavor. The nutritional value of this dish is relatively low in calories and fat, making it a healthy option for those looking for a delicious and satisfying m